# CNN Alice

## Imports et Constantes

In [18]:
############################################################## IMPORTS ET CONSTANTES ##############################################################
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import csv
import random
import zipfile
import shutil
from shutil import copyfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import * 
from tensorflow.keras import regularizers
import matplotlib
from keras.models import Sequential


from google.colab import files
#uploaded = files.upload()
#uploaded = files.upload()


training_size = 20
training_longueur = 975020
nombre_etapes_csv = 10
testing_size = 10
nb_pixels_images = 97500
largeur_image = 390
longueur_image = 250
nb_channel = 1 
n_classes = 10


############################################################## PARSING ##############################################################
def parse_data_from_input(filename, training_size, training_longueur, largeur_image, longueur_image, nb_pixels_images, n_classes):
  with open(filename) as file:
    lab = []
    img = []
  
    CSVData = open(filename)
    array = np.loadtxt(CSVData, delimiter=",")

    # récupération des labels de training
    for j in range(nb_pixels_images, training_longueur, (nb_pixels_images+2)):
        lab.append(array[j])
    labels = np.array(lab).flatten()
    
    # récupération des images de training
    for i in range(nombre_etapes_csv):
        for bigLine in range(training_size):
          for line in range(i*(nb_pixels_images+2), (i*(nb_pixels_images+2))+nb_pixels_images, 1):
            img.append((array[line][bigLine]))
        img2 = np.expand_dims(img, axis=0)
    images = img2.reshape(training_size*nombre_etapes_csv, largeur_image, longueur_image)

    return images, labels


############################################################## DATA AUGMENTATION ##############################################################
def simple_train_val_generators(training_images, training_labels, training_size, largeur_image, longueur_image, nb_channel, n_classes):
  training_images = training_images.reshape(training_size*(n_classes), largeur_image, longueur_image, nb_channel) 
  
  # data augmentation
  train_datagen = ImageDataGenerator(
       rescale = 1./255.,
       horizontal_flip=True,
       zoom_range=0.3)
  train_generator = train_datagen.flow(x=training_images,
                                       y=training_labels,
                                       batch_size=10) 

  return train_generator


############################################################## ARCHITECTURES CNNs ##############################################################
class myCallback(tf.keras.callbacks.Callback): 
  # early stoppingquand la précision sur l'ensmble d'entrainement dépasse 90% pour éviter le sur-apprentissage
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.82):
      self.model.stop_training = True

# ---------------------------------------------------------------------------------------------------------------------------------------------

def create_model(n_classes, largeur_image, longueur_image, nb_channel):
  model = tf.keras.models.Sequential([
    # features extraction
    tf.keras.layers.Conv2D(250, (2,2), activation='relu', input_shape = (largeur_image, longueur_image, nb_channel)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Dropout(0.45),
    tf.keras.layers.Conv2D(150, (2,2), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Dropout(0.45),
    tf.keras.layers.Conv2D(100, (2,2), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    # classification
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.45),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(n_classes+1, activation='softmax')
  ])
  model.summary()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

# ---------------------------------------------------------------------------------------------------------------------------------------------

def run_model(train_generator, n_classes, largeur_image, longueur_image, nb_channel, quick):
  model = create_model(n_classes, largeur_image, longueur_image, nb_channel)
  call = myCallback()
 
  def on_epoch_end(self, epoch, logs={}):
    print(logs.get('accuracy'))

  # Si quick = 0 on n'affche pas le détail de l'apprentissage
  history = model.fit(train_generator,
                        epochs=100, 
                        verbose=quick,
                        callbacks=[call])
  return model


############################################################## PREDICTIONS ET EVALUATIONS #####################################################
def prediction_1_vecteur(filename, largeur_image, longueur_image, nb_channel, indice_li, indice_co, nb_pixels_images, model, n_classes, quick):
  CSVData = open(filename)
  testing_array = np.loadtxt(CSVData, delimiter=",")
  img = []
  correct = 0
  
  # récupération de l'image à prédire
  for line in range(nb_pixels_images):
    img.append((testing_array[((nb_pixels_images+2)*indice_li)+line][indice_co]))
  img_to_pred = np.expand_dims(img, axis=0)
  img_to_pred = img_to_pred.reshape(-1,largeur_image, longueur_image, nb_channel)
  img_to_pred/=255.

  # récupération du label et du numéro de cas de l'image à prédire
  label = testing_array[((nb_pixels_images+2)*indice_li)+nb_pixels_images][indice_co]
  numero_cas_img_to_pred = testing_array[((nb_pixels_images+2)*indice_li)+nb_pixels_images+1][indice_co]
  predictions = model.predict(img_to_pred)
  classe = np.argmax(predictions, axis = 1)

  if(classe==label):
    correct=1
  else:
    correct=0
      
  if(quick==1):
    print("\nPREDICTION DES CARACTERISTIQUES A PARTIR D'1 IMAGE : ")
    print("numéro de l'image : ", label, "_", numero_cas_img_to_pred)
    print("label à prédire : ", label)
    print("classe prédite : ", classe)
    if(label==classe):
      print("La prédiction est correcte !")
      if (classe==1):
        caracteristiques = "amborella, bords lisses, phyllotaxie alternée, feuilles simples, non ligneux"
      elif (classe==2):
        caracteristiques = "castanea, bords dentés, phyllotaxie alternée, feuilles simples, ligneux"
      elif (classe==3):
        caracteristiques = "convolvulaceae, bords lisses, phyllotaxie alternée, feuilles simples, non ligneux"
      elif (classe==4):
        caracteristiques = "desmodium, bords lisses, phyllotaxie alternée, feuilles composées, non ligneux"
      elif (classe==5):
        caracteristiques = "eugenia, bords lisses, phyllotaxie opposée, feuilles simples, ligneux"
      elif (classe==6):
        caracteristiques = "laurus, bords lisses, phyllotaxie opposée, feuilles simples, ligneux"
      elif (classe==7):
        caracteristiques = "litsea, bords lisses, phyllotaxie alternée, feuilles simples, ligneux"
      elif (classe==8):
        caracteristiques = "magnolia, bords lisses, phyllotaxie alternée, feuilles simples, ligneux"
      elif (classe==9):
        caracteristiques = "monimiaceae, bords lisses, phyllotaxie oppposée, feuilles simples, ligneux"
      elif (classe==10):
        caracteristiques = "rubus, bords dentés, phyllotaxie alternée, feuilles composées, ligneux"
      elif (classe==11):
        caracteristiques = "ulmus, bords dentés, phyllotaxie alternée, feuilles simples, ligneux"
      print("caractéristiques prédites : ", caracteristiques)
    else:
      print("Malheureusement la prédiction est incorrecte...")

  return numero_cas_img_to_pred, label, classe, img_to_pred, correct

# ---------------------------------------------------------------------------------------------------------------------------------------------

def prediction_dataset(filename, testing_size, nb_pixels_images, model, n_classes, quick):
  if(quick==0):
    print("\nPREDICTION DES CARACTERISTIQUES A PARTIR DE L'ENSEMBLE DE TEST : ")  
  with open(filename) as file:
    total_good_predictions = 0
    good_predictions_per_classe = np.zeros(n_classes)
    bad_predictions_name = []
    bad_predictions_img = []
    realite = np.zeros(n_classes*testing_size)
    prediction = np.zeros(n_classes*testing_size)

    realite = realite.reshape(1,n_classes*testing_size)
    prediction = prediction.reshape(1,n_classes*testing_size)

    # parcours du dataset de test
    for indice_li in range (n_classes):                                                     
      for indice_co in range(testing_size):
        numero_cas_img_to_pred, label, classe, img_to_pred, correct = prediction_1_vecteur(filename, largeur_image, longueur_image, nb_channel, indice_li, indice_co, nb_pixels_images, model, n_classes, quick)
        realite[0][indice_li*testing_size+indice_co] = float(label)
        prediction[0][indice_li*testing_size+indice_co] = float(classe)
        if(correct==1):
          total_good_predictions+=1
          good_predictions_per_classe[indice_li]+=1
        else:
          bad_case = str(label) + '_' + str(numero_cas_img_to_pred)
          bad_predictions_name.append(bad_case)
          bad_predictions_img.append(img_to_pred)
          
    # création de la matrice de confusion
    realite = realite.reshape(n_classes*testing_size,)
    prediction = prediction.reshape(n_classes*testing_size,)
    y_actu = pd.Series(realite, name='Reality')
    y_pred = pd.Series(prediction, name='Predicted')
    df_confusion = pd.crosstab(y_actu, y_pred)
    mae = tf.keras.metrics.mean_absolute_error(realite, prediction).numpy()

    # évaluation des performances en fonction des classes à prédire
    pourcentage_good_predictions = (total_good_predictions * 100) / (n_classes*testing_size)
    if(quick==quick): # avec affichage
      print("Le pourcentage de good prédictions sur le testing set est de : ", pourcentage_good_predictions, "%. \n -------------------------------------------------------------------")
      for i in range(n_classes):
        print("Le pourcentage de good prédictions pour la classe ", str(i+1), " est de : ", (good_predictions_per_classe[i]*100)/testing_size, "%.")     
      print(df_confusion)
      print(len(bad_predictions_name), "grille(s) mal classée(s) :")
      for i in range(len(bad_predictions_name)):
        print("  ", bad_predictions_name[i])
      print("MAE : ", mae)

  return good_predictions_per_classe


## Programme principal

In [19]:
print("\n-------------\nBONJOUR, JE SUIS GULDURN et  ! \nJE SUIS UN cnn QUI VA ESTIMER LES CARACTERISTIQUES DE PLANTES A PARTIR D'IMAGES.\n-------------\n")

# IMPORTS -------------------------------------------------------------------------------------------------------
print("\nIMPORTS ...")
import random

# DATASET -------------------------------------------------------------------------------------------------------
print("\nLIENS AVEC LES DATASETS")
TRAINING_FILE = "./training.csv"
TESTING_FILE = "./testing.csv"

# PARSING -------------------------------------------------------------------------------------------------------
print("\nPARSING ... ")
training_images, training_labels = parse_data_from_input(TRAINING_FILE, training_size, training_longueur, largeur_image, longueur_image, nb_pixels_images, n_classes)
print(f"Training images has shape: {training_images.shape} and dtype: {training_images.dtype}")
print(f"Training labels has shape: {training_labels.shape} and dtype: {training_labels.dtype}")


# DATA AUGMENTATION -------------------------------------------------------------------------------------------------------
print("\nDATA AUGMENTATION")
train_generator = simple_train_val_generators(training_images, training_labels, training_size, largeur_image, longueur_image, nb_channel, n_classes)
print(f"Images of training generator have shape: {train_generator.x.shape}")
print(f"Labels of training generator have shape: {train_generator.y.shape}")


# RESEAU -------------------------------------------------------------------------------------------------------
print("\nCNN")
model = run_model(train_generator, n_classes, largeur_image, longueur_image, nb_channel, 1)


# PREDICTIONS -------------------------------------------------------------------------------------------------------
# prédiction sur 1 image de test
cas, label, classe, img, correct = prediction_1_vecteur(TESTING_FILE, largeur_image, longueur_image, nb_channel, random.randint(0,(n_classes-1)), random.randint(0,(testing_size-1)), nb_pixels_images, model, n_classes, 1)
# prédiction sur l'ensemble de test
tab_good_pred_per_classe = prediction_dataset(TESTING_FILE, testing_size, nb_pixels_images, model, n_classes, 0)




-------------
BONJOUR, JE SUIS GULDURNet  ! 
JE SUIS UN cnn QUI VA ESTIMER LES CARACTERISTIQUES DE PLANTES A PARTIR D'IMAGES.
-------------


IMPORTS ...

LIENS AVEC LES DATASETS

PARSING ... 
Training images has shape: (200, 390, 250) and dtype: float64
Training labels has shape: (200,) and dtype: float64

DATA AUGMENTATION
Images of training generator have shape: (200, 390, 250, 1)
Labels of training generator have shape: (200,)

CNN
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 389, 249, 250)     1250      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 195, 125, 250)    0         
 g2D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 195, 125, 250)     0         
         

20/20 [==============================] - 5s 202ms/step - loss: 3.8258 - accuracy: 0.0950
Epoch 2/100
20/20 [==============================] - 4s 201ms/step - loss: 2.3442 - accuracy: 0.1000
Epoch 3/100
20/20 [==============================] - 4s 202ms/step - loss: 2.3253 - accuracy: 0.0850
Epoch 4/100
20/20 [==============================] - 4s 203ms/step - loss: 2.3160 - accuracy: 0.1050
Epoch 5/100
20/20 [==============================] - 4s 203ms/step - loss: 2.3283 - accuracy: 0.1200
Epoch 6/100
20/20 [==============================] - 4s 203ms/step - loss: 2.2910 - accuracy: 0.1100
Epoch 7/100
20/20 [==============================] - 4s 204ms/step - loss: 2.2735 - accuracy: 0.1050
Epoch 8/100
20/20 [==============================] - 4s 204ms/step - loss: 2.2586 - accuracy: 0.1450
Epoch 9/100
20/20 [==============================] - 4s 204ms/step - loss: 2.3453 - accuracy: 0.1000
Epoch 10/100
20/20 [==============================] - 4s 204ms/step - loss: 2.3265 - accuracy: 0.0850
E